# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [2]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

s = 0
for sqrt_value in generator:
    s += sqrt_value

print(s)

8.382332347441762


In [3]:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [5]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person['Age'])

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
33
34
35
36
37
38


In [9]:
import duckdb
import pandas as pd

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

# Initialize a DuckDB connection
conn = duckdb.connect(database=':memory:', read_only=False)

# Load data from the first generator into a DuckDB table
people1_df = pd.DataFrame(people_1())
conn.execute("CREATE TABLE people (ID INTEGER, Name VARCHAR, Age INTEGER, City VARCHAR)")
conn.execute("INSERT INTO people SELECT * FROM people1_df")

# Calculate the sum of ages from the first generator
sum_of_ages_first_gen = conn.execute("SELECT SUM(Age) FROM people").fetchone()[0]

# Load data from the second generator and append it to the same DuckDB table
# We will alter the table to add an Occupation column before inserting the second set of data
people2_df = pd.DataFrame(people_2())
conn.execute("ALTER TABLE people ADD COLUMN Occupation VARCHAR")
conn.execute("INSERT INTO people SELECT ID, Name, Age, City, Occupation FROM people2_df")

# Calculate the sum of all ages after appending the second generator
sum_of_ages_after_append = conn.execute("SELECT SUM(Age) FROM people").fetchone()[0]

# Print out the results
print(f"Sum of ages from the first generator: {sum_of_ages_first_gen}")
print(f"Sum of ages after appending the second generator: {sum_of_ages_after_append}")

# Don't forget to close the connection when done
conn.close()


Sum of ages from the first generator: 140
Sum of ages after appending the second generator: 353


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [12]:
# Initialize a DuckDB connection
conn = duckdb.connect(database=':memory:', read_only=False)

# Create a new table with primary key ID
conn.execute("""
    CREATE TABLE people (
        ID INTEGER,
        Name VARCHAR,
        Age INTEGER,
        City VARCHAR,
        Occupation VARCHAR,
        PRIMARY KEY (ID)
    )
""")

# Load data from the first generator into the table
people1_df = pd.DataFrame(people_1())
conn.executemany("INSERT INTO people (ID, Name, Age, City) VALUES (?, ?, ?, ?)", list(people1_df.itertuples(index=False, name=None)))

# Load and merge data from the second generator into the table
people2_df = pd.DataFrame(people_2())
# Use INSERT INTO ... ON CONFLICT to replace records with the same ID
for row in people2_df.itertuples(index=False, name=None):
    conn.execute("INSERT INTO people (ID, Name, Age, City, Occupation) VALUES (?, ?, ?, ?, ?) ON CONFLICT (ID) DO UPDATE SET Name=excluded.Name, Age=excluded.Age, City=excluded.City, Occupation=excluded.Occupation", row)

# Verify the merge by selecting all records
merged_data = conn.execute("SELECT * FROM people").fetchdf()

# Output the merged data
print(merged_data)

# Calculate the sum of all ages after appending the second generator
sum_of_ages_after_append = conn.execute("SELECT SUM(Age) FROM people").fetchone()[0]
print(f"Sum of ages after appending the second generator: {sum_of_ages_after_append}")

# Close the connection
conn.close()

   ID      Name  Age    City Occupation
0   1  Person_1   26  City_A       None
1   2  Person_2   27  City_A       None
2   3  Person_3   33  City_B      Job_3
3   4  Person_4   34  City_B      Job_4
4   5  Person_5   35  City_B      Job_5
5   6  Person_6   36  City_B      Job_6
6   7  Person_7   37  City_B      Job_7
7   8  Person_8   38  City_B      Job_8
Sum of ages after appending the second generator: 266


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX